In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OR_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14
1,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14
2,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14
3,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14
4,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14
1,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14
2,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14
3,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14
4,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
919,9284477302,OR,"4530 Mahoney Rd NE, Saint Paul, OR",45.153780,-122.971550,NaN
938,2245188507,OR,"1000 SW Broadway, Suite 1800, PORTLAND, OR",45.516927,-122.680657,NaN
1093,4436777110,OR,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1757,7531057707,OR,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
3253,6623328301,OR,"10090 SE WASHINGTON, PORTLAND, OR",45.518143,-122.559820,NaN
4834,7689617110,OR,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
4948,4186547107,OR,"100 C Street, Phoenix, OR",42.269568,-122.819262,NaN
5301,3602737100,OR,"4233 SE 182ND AVE # 263, GRESHAM, OR",45.491901,-122.475992,NaN
6780,7248847103,OR,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
7797,9965578408,OR,"PO BOX 3070 APT 324, OREGON CITY, OR",45.355610,-122.605853,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('41')] #OR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/19 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

19


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12403/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
919,9284477302,OR,"4530 Mahoney Rd NE, Saint Paul, OR",45.153780,-122.971550,[410470101001083]
938,2245188507,OR,"1000 SW Broadway, Suite 1800, PORTLAND, OR",45.516927,-122.680657,[410510106022047]
1093,4436777110,OR,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,[]
1757,7531057707,OR,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,[]
3253,6623328301,OR,"10090 SE WASHINGTON, PORTLAND, OR",45.518143,-122.559820,[410510082011012]
4834,7689617110,OR,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,[]
4948,4186547107,OR,"100 C Street, Phoenix, OR",42.269568,-122.819262,[410290016025028]
5301,3602737100,OR,"4233 SE 182ND AVE # 263, GRESHAM, OR",45.491901,-122.475992,[410510098034000]
6780,7248847103,OR,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,[]
7797,9965578408,OR,"PO BOX 3070 APT 324, OREGON CITY, OR",45.355610,-122.605853,[410050224004028]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12403/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
919,9284477302,OR,"4530 Mahoney Rd NE, Saint Paul, OR",45.153780,-122.971550,410470101001083
938,2245188507,OR,"1000 SW Broadway, Suite 1800, PORTLAND, OR",45.516927,-122.680657,410510106022047
1093,4436777110,OR,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1757,7531057707,OR,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
3253,6623328301,OR,"10090 SE WASHINGTON, PORTLAND, OR",45.518143,-122.559820,410510082011012
4834,7689617110,OR,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
4948,4186547107,OR,"100 C Street, Phoenix, OR",42.269568,-122.819262,410290016025028
5301,3602737100,OR,"4233 SE 182ND AVE # 263, GRESHAM, OR",45.491901,-122.475992,410510098034000
6780,7248847103,OR,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
7797,9965578408,OR,"PO BOX 3070 APT 324, OREGON CITY, OR",45.355610,-122.605853,410050224004028


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1093,4436777110,OR,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1757,7531057707,OR,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
4834,7689617110,OR,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
6780,7248847103,OR,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
10646,5541307105,OR,"34747 Hwy 101, ASTORIA, OR",41.998161,-124.208246,NaN
11686,4528678304,OR,"2303 NW York St, 2303 NW York St, OR",43.299428,-74.217933,NaN
11736,3292878401,OR,"1200 Highland Ave, Enterprise, OR",32.674723,-117.099477,NaN
11794,3313707106,OR,"137 E Main Street, LOWELL, OR",42.933643,-85.341961,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
919,9284477302,410470101001083
938,2245188507,410510106022047
1093,4436777110,NaN
1757,7531057707,NaN
3253,6623328301,410510082011012
4834,7689617110,NaN
4948,4186547107,410290016025028
5301,3602737100,410510098034000
6780,7248847103,NaN
7797,9965578408,410050224004028


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,720036,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14
1,720037,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14
2,720038,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14
3,720039,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14
4,720040,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,720036,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14,NaN
1,720037,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14,NaN
2,720038,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14,NaN
3,720039,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14,NaN
4,720040,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14,NaN
...,...,...,...,...,...,...,...,...
13398,733434,9768048305,OR,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,4.105100e+14,NaN
13399,733435,9833377202,OR,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,4.106703e+14,NaN
13400,733436,9863097801,OR,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,4.100502e+14,NaN
13401,733437,9919177006,OR,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,4.101700e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,720036,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,4.105101e+14,NaN,410510059032002.0
1,720037,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,4.106703e+14,NaN,410670316212001.0
2,720038,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,4.105101e+14,NaN,410510106022062.0
3,720039,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,4.101913e+14,NaN,410191300012008.0
4,720040,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,4.103900e+14,NaN,410390035001001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1199347308,OR,"5550 SW Macadam Ave, Suite 200, PORTLAND, OR",45.483919,-122.672891,410510059032002.0
1,2346797200,OR,"14747 NW Greenbrier Pkwy, BEAVERTON, OR",45.525781,-122.828285,410670316212001.0
2,2507148910,OR,"1211 SW 5th Ave, Portland, OR",45.515341,-122.680144,410510106022062.0
3,4687487710,OR,"2635 OLD HWY 99 S, ROSEBURG, OR",43.182433,-123.367436,410191300012008.0
4,4842197707,OR,"305 S. 4TH STREET, SPRINGFIELD, OR",44.043423,-123.020368,410390035001001.0
...,...,...,...,...,...,...
13398,9768048305,OR,"1204 NW 21st Ave, Portland, OR",45.531431,-122.694377,410510049013002.0
13399,9833377202,OR,"18515 WEST UNION RD, PORTLAND, OR",45.558244,-122.867883,410670315091021.0
13400,9863097801,OR,"8620 SE 347TH AVE, BORING, OR",45.461039,-122.305030,410050234011005.0
13401,9919177006,OR,"1506 NE Willianson Blvd, BEND, OR",44.064700,-121.269801,410170018012001.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1093,4436777110,OR,"350 N. MAIN ST, LEXINGTON, OR",42.490979,-83.144072,NaN
1757,7531057707,OR,"1 ABBEY DR, ST BENEDICT, OR",41.514392,-87.845932,NaN
4834,7689617110,OR,"12178 GARDEN PL SW, TIGARD, OR",51.919438,19.145136,NaN
6780,7248847103,OR,"43150 Hwy 101, PORT ORFORD, OR",39.661889,-123.481011,NaN
10646,5541307105,OR,"34747 Hwy 101, ASTORIA, OR",41.998161,-124.208246,NaN
11686,4528678304,OR,"2303 NW York St, 2303 NW York St, OR",43.299428,-74.217933,NaN
11736,3292878401,OR,"1200 Highland Ave, Enterprise, OR",32.674723,-117.099477,NaN
11794,3313707106,OR,"137 E Main Street, LOWELL, OR",42.933643,-85.341961,NaN


Convert to File

In [19]:
fips_OR = fips_merge_drop

In [20]:
fips_OR.to_csv("../../../data/state_data/geo/geo_fips/23/OR_fips_scraped.csv") 